In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('order_brush_order.csv')
data.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [3]:
data['event_time'] = data['event_time'].apply(lambda x: pd.Timestamp(x).timestamp())
data.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,1.577406e+09
1,31118059853484,156423439,46057927,1.577448e+09
2,31123355095755,173699291,67341739,1.577453e+09
3,31122059872723,63674025,149380322,1.577452e+09
4,31117075665123,127249066,149493217,1.577447e+09


In [4]:
data = data.drop(columns='orderid')
data = data.sort_values(by=['shopid', 'event_time'])
data.head(10)

,shopid,userid,event_time
59,10009,196962305,1.577416e+09
76187,10051,2854032,1.577474e+09
9055,10051,48600461,1.577585e+09
188513,10061,168750452,1.577525e+09
174620,10061,194819216,1.577535e+09
200439,10061,130633421,1.577742e+09
145426,10061,62464559,1.577761e+09
167859,10084,4401933,1.577405e+09
178051,10084,13837190,1.577407e+09
91837,10084,39828049,1.577409e+09


In [53]:
arr = data.values
out = dict()
temp = dict()
for row in range(len(arr)):
    out[arr[row][0]] = 0
for row in range(len(arr)):
    user = dict()
    user[arr[row][1]] = user.get(arr[row][1], 0) + 1
    order = row + 1
    while order < len(arr) and arr[order][0] == arr[row][0] and arr[order][2] <= arr[row][2] + 3600:
        user[arr[order][1]] = user.get(arr[order][1], 0) + 1
        order += 1
    score = (order - row) / len(user)
    user = dict(sorted(user.items(), key=lambda kv: kv[1], reverse=True))
    s = 0
    if score >= 3:
        for k, v in user.items():
            s += v
        for k, v in user.items():
            first = k
            break
        for k, v in user.items():
            if v == user[first]:
                temp[k] = v / s
    if row + 1 < len(arr) and arr[row][0] != arr[row + 1][0] and len(temp) > 0:
        temp = dict(sorted(temp.items(), key=lambda kv: kv[1], reverse=True))
        for k, v in temp.items():
            first = k
            break
        user_arr = list()
        for k, v in temp.items():
            if v == temp[first]:
                user_arr.append(int(k))
        user_arr = sorted(user_arr)
        for i in user_arr:
            out[arr[row][0]] = f'{out[arr[row][0]]}&{i}'
        out[arr[row][0]] = out[arr[row][0]][2:]
        temp = dict()
output = list()
for k, v in out.items():
    output.append([int(k), v])
pd.DataFrame(output).to_csv('submission.csv', header=['shopid', 'userid'], index=None)
    